<a href="https://colab.research.google.com/github/MWFK/Nike_Case_Assignments/blob/main/1.%20VIP_Promotion_Python_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

In [3]:
# Data source: Sales domain
# One month's data

path_transactions = '/content/transactions.parquet'
df_transactions   = pq.read_table(path_transactions)
df_transactions   = df_transactions.to_pandas()

# print(df_transactions.shape)
# print(df_transactions.dtypes)
# df_transactions

# Data source: Customer domain
# Two week's data
# Date format yyyy-mm-dd

path_vips_1101 = '/content/vips_2020-11-01.csv'
df_vips_1101   = pd.read_csv (path_vips_1101)

# print(df_vips_1101.shape)
# print(df_vips_1101.dtypes)
# df_vips_1101.head()

# Data source: Customer domain
# One week's data

path_vips_1115 = '/content/vips_2020-11-15.csv'
df_vips_1115   = pd.read_csv (path_vips_1115)

# print(df_vips_1115.shape)
# print(df_vips_1115.dtypes)
# df_vips_1115.head()

# Data source: Customer domain
# One week's data

path_vips_1125 = '/content/vips_2020-11-25.csv'
df_vips_1125   = pd.read_csv (path_vips_1125)

# print(df_vips_1125.shape)
# print(df_vips_1125.dtypes)
# df_vips_1125.head()

# Source: Marketing domain
# One month's  data
# Date format dd-mm-yyyy
'''
'meta_change_date' have happened in 04-01-2020 and 25-01-2020, 
VIP's data have occured on 01-11-2020, 15-11-2020, 25-11-2020, 
Which means all the VIP data files needs to be updated according 
to he mapping file, before using them. 
'''

path_vips_mapping = '/content/umd_vip_to_profile_mapping.csv'
df_vips_mapping = pd.read_csv(path_vips_mapping, sep=';')

# print(df_vips_mapping.shape)
# print(df_vips_mapping.dtypes)
# df_vips_mapping.head()

In [7]:
def vip_process(df_vips_1101, df_vips_mapping):

  # Only keep VIP's currently based in the Netherlands
  df_vips_1101 = df_vips_1101.loc[df_vips_1101['country'] == 'The Netherlands'].reset_index(drop=True)
  
  ''' 
  An athlete got his status active as no since he was duplicated in the system
  and since from the meta_commets we have no other indication, that means he still an active VIP, 
  so we'll delete that duplicate VIP ID with active status no
  '''
  df_vips_mapping.drop_duplicates(subset='vip_id', keep="first", inplace=True)

  # Merge df_vips_mapping with df_vips_1101 using 'vip_id'
  df_vips_1101_mapped = pd.merge(df_vips_1101, df_vips_mapping, on='vip_id', how='left').reset_index(drop=True)

  # Only keep active VIP's
  df_vips_1101_mapped = df_vips_1101_mapped.loc[df_vips_1101_mapped['active'] == 'yes'].reset_index(drop=True)

  return df_vips_1101_mapped[['profile_id', 'first_name', 'email']]

In [27]:
vip_process(df_vips_1115, df_vips_mapping)

,profile_id,first_name,email
0,1-5151-4642-1,Brandon,Brandon.Ritter@nike.com
1,1-9860-3438-0,Kristin,Kristin@Brooks.com
2,1-59971-953-3,Seth,Seth@Lee.com


In [39]:
def transaction_process(df_transactions, start_date, end_date):

  # Filter transactions dates(between 2020-11-01 and 2020-11-14) 
  df_transactions_1101 = df_transactions[(df_transactions['transaction_date'] >= start_date) & (df_transactions['transaction_date'] < end_date)]

  # Drop the cancelled items
  df_transactions_1101 = df_transactions_1101.loc[df_transactions_1101['cancellation_flag'] != 'yes'].reset_index(drop=True)

  '''
  discount_amount_per_unit values should be prepapred to calculate the unit price after discount so if it's:
  NaN it should be changed with the value 1.
  100.0 should be 0.
  The rest should be divided by 100.
  '''
  df_transactions_1101['discount_amount_per_unit'] = np.where(df_transactions_1101['discount_amount_per_unit'] == 100, 0, df_transactions_1101['discount_amount_per_unit'])
  df_transactions_1101['discount_amount_per_unit'] = np.where(df_transactions_1101['discount_amount_per_unit'] >= 0, df_transactions_1101['discount_amount_per_unit']/100, 1)

  # Calculate the unit price after applying the discount
  df_transactions_1101['unit_price_after_discount'] = df_transactions_1101['recommended_retail_price_per_unit'].abs() * df_transactions_1101['discount_amount_per_unit'] * df_transactions_1101['quantity']

  # Filter out unecessary columns
  df_transactions_1101 = df_transactions_1101[['profile_id', 'unit_price_after_discount']]

  return df_transactions_1101.groupby(['profile_id']).sum()

In [40]:
start_date = '2020-11-15'
end_date   = '2020-11-25'
transaction_process(df_transactions, start_date, end_date)

,unit_price_after_discount
profile_id,
1-4786-9297-9,644.00
1-5151-4642-1,1550.75
1-57006-547-0,755.00


In [41]:
def vip_sales_overview(df_vips_1101, df_vips_mapping, df_transactions, start_date, end_date):
  df_vips_1101_mapped  = vip_process(df_vips_1101, df_vips_mapping)
  df_transactions_1101 = transaction_process(df_transactions, start_date, end_date)
  df_vips_1101_mapped_transactions = pd.merge(df_vips_1101_mapped, df_transactions_1101, on='profile_id', how='left').reset_index(drop=True)
  return df_vips_1101_mapped_transactions[['profile_id','first_name', 'email', 'unit_price_after_discount']]

In [42]:
start_date = '2020-11-01'
end_date   = '2020-11-15'
vip_sales_overview(df_vips_1101, df_vips_mapping, df_transactions, start_date, end_date)

'\nInterpretations\nBrandon did not make a purchase during the first two weeks.\nKristin have bought two of the same unit during the first two weeks, \nThen he returned one of them during the last week \nTherefore, we should keep the minus values as an indications for returned units.\n'

'''
Interpretations
Brandon did not make a purchase during the first two weeks.
Kristin have bought two of the same unit during the first two weeks, 
Then he returned one of them during the last week 
Therefore, we should keep the minus values as an indications for returned units.
'''

In [45]:
start_date = '2020-11-15'
end_date   = '2020-11-25'
vip_sales_overview(df_vips_1115, df_vips_mapping, df_transactions, start_date, end_date)

,profile_id,first_name,email,unit_price_after_discount
0,1-5151-4642-1,Brandon,Brandon.Ritter@nike.com,1550.75
1,1-9860-3438-0,Kristin,Kristin@Brooks.com,NaN
2,1-59971-953-3,Seth,Seth@Lee.com,NaN


Seth did not make any transactions during this period.

Kristin did not make any transactions during this period.

In [44]:
start_date = '2020-11-25'
end_date   = '2020-11-31'
vip_sales_overview(df_vips_1115, df_vips_mapping, df_transactions, start_date, end_date)

,profile_id,first_name,email,unit_price_after_discount
0,1-5151-4642-1,Brandon,Brandon.Ritter@nike.com,372.0
1,1-9860-3438-0,Kristin,Kristin@Brooks.com,-181.0
2,1-59971-953-3,Seth,Seth@Lee.com,NaN


Seth did not make any transactions during this period.

Kristin have returned on eof the two items that he have purchased in the first week of november.


In [34]:
df_transactions

,store,transaction_date,order_number,line_number,product_id,product_name,quantity,currency,recommended_retail_price_per_unit,discount_amount_per_unit,profile_id,cancellation_flag,cancellation_date,source_system
0,.com,2020-11-20,422681959244,1,720854836340997,zJnSpFriDYyNDJhwxMxD,1,NL,721,NaN,1-5151-4642-1,None,None,DIGITAL
1,.com,2020-11-20,422681959244,2,999789223355744,XVnSxtFOJUDGlfxVQauS,1,NL,71,25.0,1-5151-4642-1,None,None,DIGITAL
2,.com,2020-11-20,422681959244,3,343125809243217,XkzpDAMIbZmTvuMTmGPU,1,NL,812,NaN,1-5151-4642-1,None,None,DIGITAL
3,.com,2020-11-23,982990828489,1,451789186646625,PeSsoTndtARHiWxgEooV,1,SDG,755,NaN,1-57006-547-0,None,None,DIGITAL
4,.com,2020-11-28,560607850517,1,162643898729888,XidxwPxWZGzNaZqPosjV,1,MOP,372,NaN,1-5151-4642-1,None,None,DIGITAL
5,.com,2020-11-27,982990828489,1,451789186646625,PeSsoTndtARHiWxgEooV,-1,SDG,-755,NaN,1-57006-547-0,None,None,DIGITAL
6,.com,2020-11-06,255329180770,1,89920978863916,PCZZxsBhrPPbFXGHgisb,1,GMD,678,NaN,1-4947-2509-6,None,None,DIGITAL
7,.com,2020-11-06,255329180770,2,28756773917332,AEzisXHrWPYPdGSKhASe,1,GMD,678,NaN,1-4947-2509-6,None,None,DIGITAL
8,.com,2020-11-06,255329180770,3,408285286501930,ADUSDiRfHjyQFwhnQJiL,2,GMD,400,NaN,1-4947-2509-6,None,None,DIGITAL
9,.com,2020-11-06,255329180770,3,408285286501930,ADUSDiRfHjyQFwhnQJiL,2,GMD,400,NaN,1-4947-2509-6,yes,2020-11-06,DIGITAL


In [37]:
df_vips_1125 

,vip_id,first_name,last_name,vip_type,country,email
0,4544056563,Brandon,Ritter,Athlete + Donator,The Netherlands,Brandon.Ritter@nike.com
1,9339003839,Adam,Hall,Athlete,Czech Republic,Adam.Hall@nike.com
2,1600627714,Fernando,Ford,Donator,Australia,Fernando@Ford.com
3,7531485682,Edwin,Floyd,Donator,The Netherlands,Edwin.Floyd@nike.com
4,4271215441,Kristin,Brooks-Ritter,Donator,The Netherlands,Kristin@Brooks.com
5,1154401046,Justin,Powell,Athlete,Finland,Justin.Powell@nike.com
6,4262242729,William,Smith,Athlete,United States,William.Smith@nike.com
7,6159265415,Angela,Trujillo,Donator,Belarus,Angela@Trujillo.com
8,4711973834,Seth,Lee,Donator,The Netherlands,Seth@Lee.com


In [38]:
df_vips_mapping

,vip_id,profile_id,active,meta_change_date,meta_comments
0,4544056563,1-5151-4642-1,yes,NaN,NaN
2,9339003839,1-4786-9297-9,yes,NaN,NaN
3,1600627714,1-4947-2509-6,yes,NaN,NaN
4,7531485682,1-57006-547-0,no,25-1-2020,No longer a donator
5,4271215441,1-9860-3438-0,yes,NaN,NaN
6,1154401046,1-85886-104-7,yes,NaN,NaN
7,4262242729,0-89923-575-1,yes,NaN,NaN
8,6159265415,0-10-241138-7,yes,NaN,NaN
9,4711973834,1-59971-953-3,yes,NaN,NaN
